In [4]:
%pip install 'chronos-forecasting>=2.0' 'pandas[pyarrow]' 'matplotlib' 'python-dotenv' --quiet

In [5]:
from dotenv import load_dotenv

load_dotenv('../.env')

False

In [ ]:
import os

# Use only 1 GPU if available
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ["HF_TOKEN"] =  ...

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from chronos import BaseChronosPipeline, Chronos2Pipeline
from IPython.display import clear_output

# Load the Chronos-2 pipeline
# GPU recommended for faster inference, but CPU is also supported
pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-2", 
    # device_map="cuda"
        device_map="cpu"
        )

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
df = pd.read_csv('https://www.dropbox.com/scl/fi/s83jig4zews1xz7vhezui/allDataWithCalculatedColumns.csv?rlkey=9mm4zwaugxyj2r4ooyd39y4nl&raw=1')
df.ds = pd.to_datetime(df.ds, errors="coerce")
df['id']='jgh'
df.tail()

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM,total_tbs,vert_tbs,pod_tbs,overflow,id
42861,2025-11-25 06:00:00,5,20,1,7,6,27,1,3,0,...,1,0,0,14,9,14,10,4,10,jgh
42862,2025-11-25 07:00:00,1,21,1,8,2,29,0,3,0,...,1,0,0,14,9,13,9,4,10,jgh
42863,2025-11-25 08:00:00,4,25,2,10,6,35,2,5,0,...,1,0,0,15,9,12,10,2,9,jgh
42864,2025-11-25 09:00:00,11,36,9,19,20,55,0,5,0,...,1,0,0,15,9,11,10,1,9,jgh
42865,2025-11-25 10:00:00,7,43,8,27,15,70,3,8,0,...,1,0,0,16,9,15,12,3,9,jgh


In [8]:
import pandas as pd
from pandas.tseries.frequencies import to_offset
from IPython.display import clear_output

ID_COL = "id"
TS_COL = "ds"
TARGETS = ['total_tbs']

df = df.copy()
df[TS_COL] = pd.to_datetime(df[TS_COL], errors="coerce")
df = df.dropna(subset=[TS_COL])

# Snap to exact hours (lowercase 'h' to avoid FutureWarning)
df[TS_COL] = df[TS_COL].dt.floor("h")

# Sort + dedupe
df = df.sort_values([ID_COL, TS_COL]).drop_duplicates([ID_COL, TS_COL], keep="last")

def regularize_hourly(g: pd.DataFrame) -> pd.DataFrame:
    """
    Reindex each group's timestamps to strict hourly and fill gaps.
    Works whether the grouping column is present or omitted (include_groups=False).
    """
    # The group key (id) is available as g.name; if ID_COL exists, prefer it.
    sid = g[ID_COL].iloc[0] if ID_COL in g.columns else g.name

    g = g.sort_values(TS_COL)
    full_idx = pd.date_range(g[TS_COL].min(), g[TS_COL].max(), freq="h")
    g = g.set_index(TS_COL).reindex(full_idx)
    g.index.name = TS_COL

    # restore id (constant for the whole group)
    g[ID_COL] = sid

    # numeric + fill for targets
    for col in TARGETS:
        if col in g.columns:
            g[col] = pd.to_numeric(g[col], errors="coerce").ffill().bfill()
    return g.reset_index()

# Call apply with include_groups=False if supported; else fall back
gb = df.groupby(ID_COL, group_keys=False)
try:
    df = gb.apply(regularize_hourly, include_groups=False)
except TypeError:
    # older pandas without include_groups
    df = gb.apply(regularize_hourly)

# Assert truly hourly (accept 'h' and 'H')
g = df[df[ID_COL] == "jgh"].sort_values(TS_COL)
freq = pd.infer_freq(g[TS_COL])
if not freq:
    raise ValueError("No inferable frequency after regularization.")
if to_offset(freq).name.lower() != "h":
    # extra check independent of infer_freq
    diffs = g[TS_COL].diff().dropna()
    bad = g.loc[diffs != pd.Timedelta(hours=1), TS_COL].head(10).tolist()
    raise ValueError(f"Non-1h gaps remain around: {bad}")



In [9]:
# Predict
pred_df = pipeline.predict_df(
    df,
    prediction_length=24,
    quantile_levels=[0.1, 0.5, 0.9],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)
pred_df

,id,ds,target_name,predictions,0.1,0.5,0.9
0,jgh,2025-11-25 11:00:00,total_tbs,19.753437,15.721951,19.753437,23.749315
1,jgh,2025-11-25 12:00:00,total_tbs,25.029087,18.912363,25.029087,31.435581
2,jgh,2025-11-25 13:00:00,total_tbs,32.323257,23.889149,32.323257,40.106224
3,jgh,2025-11-25 14:00:00,total_tbs,35.319305,25.685329,35.319305,44.433418
4,jgh,2025-11-25 15:00:00,total_tbs,38.139164,27.355043,38.139164,47.595650
5,jgh,2025-11-25 16:00:00,total_tbs,41.407425,29.648672,41.407425,51.472557
6,jgh,2025-11-25 17:00:00,total_tbs,38.304260,25.579184,38.304260,49.484268
7,jgh,2025-11-25 18:00:00,total_tbs,35.639309,23.134516,35.639309,48.234520
8,jgh,2025-11-25 19:00:00,total_tbs,33.396877,21.155796,33.396877,45.446354
9,jgh,2025-11-25 20:00:00,total_tbs,30.512127,18.702192,30.512127,43.052414


In [12]:
#lelt's perform cross-validation to evaluate our model's performance
#start with df.ds ending on 2025-01-01 08:00:00 and predict the next 8 hours
#then compare with actuals
cutoff_date = pd.Timestamp("2025-01-01 08:00:00")
history_df = df[df[TS_COL] <= cutoff_date]
forecast_df = pipeline.predict_df(
    history_df,
    prediction_length=8,
    quantile_levels=[0.5],
    id_column=ID_COL,
    timestamp_column=TS_COL,
    target=TARGETS,
)
# Merge forecasts with actuals
actuals_df = df[(df[TS_COL] > cutoff_date) & (df[TS_COL] <= cutoff_date + pd.Timedelta(hours=8))]
merged_df = pd.merge(forecast_df, actuals_df, on=[ID_COL, TS_COL], suffixes=('_pred', '_actual'))
# Calculate RMSE for the predictions
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(
    merged_df['total_tbs'],
    merged_df['predictions']
)
print(f"RMSE for the 8-hour forecast: {rmse}")



RMSE for the 8-hour forecast: 37.92843462412088


In [33]:
#now use tqdm to iterate through a new forecast every following day starting at 2025-01-01 always at 8am and average out the RMSEs
from tqdm.notebook import tqdm
rmse_list = []
start_date = pd.Timestamp("2025-01-01 16:00:00")
end_date = df[TS_COL].max() - pd.Timedelta(hours=8)
# end_date = pd.Timestamp("2025-01-05 08:00:00")
for _ in tqdm(range((end_date - start_date).days)):
    history_df = df[df[TS_COL] <= start_date]
    # print(history_df.ds.max())
    forecast_df = pipeline.predict_df(
        history_df,
        prediction_length=8,
        quantile_levels=[0.5],
        id_column=ID_COL,
        timestamp_column=TS_COL,
        target=TARGETS,
    )
    actuals_df = df[(df[TS_COL] > start_date) & (df[TS_COL] <= start_date + pd.Timedelta(hours=8))]
    merged_df = pd.merge(forecast_df, actuals_df, on=[ID_COL, TS_COL], suffixes=('_pred', '_actual'))
    rmse = mean_squared_error(
        merged_df['total_tbs'],
        merged_df['predictions']
    )
    rmse_list.append(rmse)
    print(rmse)
    start_date += pd.Timedelta(days=1)
average_rmse = sum(rmse_list) / len(rmse_list)
print(f"Average RMSE over the period: {average_rmse}")

  0%|          | 0/316 [00:00<?, ?it/s]

73.4963992215985
42.994789491468055
327.08347856883347
60.68348983573014
18.60393970670475
146.21347762184814
16.335154826307644
34.34325724865221
121.41309804913408
11.840215114093553
42.619944982237485
50.9739136718963
55.21406163546408
117.35878362394487
24.68718893283858
37.3123610247676
117.84111315494829
62.31707148425312
19.56962495198877
133.80616573396856
63.62354144688152
62.59166608917076
49.357523911707176
15.663567579581468
69.6897081606553
51.52362971958928
93.5806795242197
35.48292807676171
118.57451550070505
13.71307589887681
208.52038020967575
3.1508102989082545
97.09296838291277
9.88177953711147
55.16259507627001
11.193362669328508
9.743788514928838
12.588659648528392
170.71569699996917
15.137724285215427
33.170903200249995
20.956503264008916
22.325393279591026
181.86535522524946
15.672107035089539
130.45207771543164
186.25399282183582
6.42702484785309
7.211903344056054
16.164402460513884
27.86681043197632
81.18996351684928
58.112869771365695
20.36766856916529
27.2346

In [24]:
#I want to know the average RMSE by day of week (e.g., is the model worse on mondays than on fridays?)
import collections
rmse_by_day = collections.defaultdict(list)
start_date = pd.Timestamp("2025-01-01 08:00:00")
end_date = pd.Timestamp("2025-04-01 08:00:00")
for _ in tqdm(range((end_date - start_date).days)):
    history_df = df[df[TS_COL] <= start_date]
    forecast_df = pipeline.predict_df(
        history_df,
        prediction_length=8,
        quantile_levels=[0.5],
        id_column=ID_COL,
        timestamp_column=TS_COL,
        target=TARGETS,
    )
    actuals_df = df[(df[TS_COL] > start_date) & (df[TS_COL] <= start_date + pd.Timedelta(hours=8))]
    merged_df = pd.merge(forecast_df, actuals_df, on=[ID_COL, TS_COL], suffixes=('_pred', '_actual'))
    rmse = mean_squared_error(
        merged_df['total_tbs'],
        merged_df['predictions']
    )
    day_of_week = start_date.day_name()
    rmse_by_day[day_of_week].append(rmse)
    start_date += pd.Timedelta(days=1)
# Calculate average RMSE by day of week
average_rmse_by_day = {day: sum(rmses) / len(rmses) for day, rmses in rmse_by_day.items()}
print("Average RMSE by day of week:")
for day, avg_rmse in average_rmse_by_day.items():
    print(f"{day}: {avg_rmse}")

  0%|          | 0/90 [00:00<?, ?it/s]

Average RMSE by day of week:
Wednesday: 41.38582932147818
Thursday: 95.91189453069525
Friday: 54.82910380477245
Saturday: 70.59074537012046
Sunday: 63.84156222594706
Monday: 64.8229408294746
Tuesday: 68.85372658078583


In [17]:
#add day of the week as a feature and see if it improves performance
df['day_of_week'] = df[TS_COL].dt.dayofweek

#now use tqdm to iterate through a new forecast every following day starting at 2025-01-01 always at 8am and average out the RMSEs
from tqdm.notebook import tqdm
rmse_list = []
start_date = pd.Timestamp("2025-01-01 08:00:00")
# end_date = df[TS_COL].max() - pd.Timedelta(hours=8)
end_date = pd.Timestamp("2025-02-01 08:00:00")
for _ in tqdm(range((end_date - start_date).days)):
    print(start_date)
    history_df = df[df[TS_COL] <= start_date]
    forecast_df = pipeline.predict_df(
        history_df,
        prediction_length=8,
        quantile_levels=[0.5],
        id_column=ID_COL,
        timestamp_column=TS_COL,
        target=TARGETS,
    )
    actuals_df = df[(df[TS_COL] > start_date) & (df[TS_COL] <= start_date + pd.Timedelta(hours=8))]
    merged_df = pd.merge(forecast_df, actuals_df, on=[ID_COL, TS_COL], suffixes=('_pred', '_actual'))
    rmse = mean_squared_error(
        merged_df['total_tbs'],
        merged_df['predictions']
    )
    rmse_list.append(rmse)
    start_date += pd.Timedelta(days=1)
    #clear output
    clear_output(wait=True)
average_rmse = sum(rmse_list) / len(rmse_list)
print(f"Average RMSE over the period: {average_rmse}")

Average RMSE over the period: 78.45709604316224


In [20]:
#add holidays as a feature and see if it improves performance
import pandas as pd
import holidays

def add_holiday_flags(
    df: pd.DataFrame,
    ts_col: str = "ds",
    local_tz: str = "America/Montreal",
    observed: bool = True,
    include_names: bool = False,
) -> pd.DataFrame:
    """
    Adds boolean columns:
      • is_qc_holiday       — Québec public holiday (CA-QC)
      • is_jewish_holiday   — Israeli public/Jewish holiday (IL)
    Optionally adds:
      • qc_holiday_name
      • jewish_holiday_name

    Notes:
      • Holiday checks are date-based (00:00–24:00 local calendar date),
        not sundown-to-sundown observance.
      • NaT timestamps are ignored gracefully.
    """
    out = df.copy()

    # 1) Parse to datetime
    out[ts_col] = pd.to_datetime(out[ts_col], errors="coerce")

    # 2) Get the calendar DATE to use for holiday lookup
    #    - If tz-aware: convert to Montreal then take .date
    #    - If naive: assume values already represent local Montreal wall-clock; just take .date
    if getattr(out[ts_col].dt, "tz", None) is not None:
        dates_for_calendar = out[ts_col].dt.tz_convert(local_tz).dt.date
    else:
        dates_for_calendar = out[ts_col].dt.date

    # 3) Build a SAFE integer year range for the holiday objects
    years_series = pd.Series(dates_for_calendar)
    years_series = years_series.dropna().map(lambda d: int(pd.Timestamp(d).year))
    if years_series.empty:
        raise ValueError("No valid datetimes found to extract holiday years.")
    years = list(range(int(years_series.min()), int(years_series.max()) + 1))

    # 4) Construct holiday calendars
    qc_holidays = holidays.Canada(subdiv="QC", years=years, observed=observed)
    il_holidays = holidays.Israel(years=years, observed=observed)

    # 5) Flag membership
    out["is_qc_holiday"] = [ (d in qc_holidays) if pd.notna(pd.Timestamp(d)) else False
                             for d in dates_for_calendar ]
    # out["is_jewish_holiday"] = [ (d in il_holidays) if pd.notna(pd.Timestamp(d)) else False
    #                              for d in dates_for_calendar ]

    if include_names:
        out["qc_holiday_name"] = [ qc_holidays.get(d, "") if pd.notna(pd.Timestamp(d)) else ""
                                   for d in dates_for_calendar ]
        out["jewish_holiday_name"] = [ il_holidays.get(d, "") if pd.notna(pd.Timestamp(d)) else ""
                                       for d in dates_for_calendar ]

    return out

df = pd.read_csv('https://www.dropbox.com/scl/fi/s83jig4zews1xz7vhezui/allDataWithCalculatedColumns.csv?rlkey=9mm4zwaugxyj2r4ooyd39y4nl&raw=1')
df.ds = pd.to_datetime(df.ds, errors="coerce")
df['id']='jgh'
df = add_holiday_flags(df, ts_col=TS_COL, include_names=False)

ID_COL = "id"
TS_COL = "ds"
TARGETS = ['total_tbs']

df = df.copy()
df[TS_COL] = pd.to_datetime(df[TS_COL], errors="coerce")
df = df.dropna(subset=[TS_COL])

# Snap to exact hours (lowercase 'h' to avoid FutureWarning)
df[TS_COL] = df[TS_COL].dt.floor("h")

# Sort + dedupe
df = df.sort_values([ID_COL, TS_COL]).drop_duplicates([ID_COL, TS_COL], keep="last")

def regularize_hourly(g: pd.DataFrame) -> pd.DataFrame:
    """
    Reindex each group's timestamps to strict hourly and fill gaps.
    Works whether the grouping column is present or omitted (include_groups=False).
    """
    # The group key (id) is available as g.name; if ID_COL exists, prefer it.
    sid = g[ID_COL].iloc[0] if ID_COL in g.columns else g.name

    g = g.sort_values(TS_COL)
    full_idx = pd.date_range(g[TS_COL].min(), g[TS_COL].max(), freq="h")
    g = g.set_index(TS_COL).reindex(full_idx)
    g.index.name = TS_COL

    # restore id (constant for the whole group)
    g[ID_COL] = sid

    # numeric + fill for targets
    for col in TARGETS:
        if col in g.columns:
            g[col] = pd.to_numeric(g[col], errors="coerce").ffill().bfill()
    return g.reset_index()

# Call apply with include_groups=False if supported; else fall back
gb = df.groupby(ID_COL, group_keys=False)
try:
    df = gb.apply(regularize_hourly, include_groups=False)
except TypeError:
    # older pandas without include_groups
    df = gb.apply(regularize_hourly)

# Assert truly hourly (accept 'h' and 'H')
g = df[df[ID_COL] == "jgh"].sort_values(TS_COL)
freq = pd.infer_freq(g[TS_COL])
if not freq:
    raise ValueError("No inferable frequency after regularization.")
if to_offset(freq).name.lower() != "h":
    # extra check independent of infer_freq
    diffs = g[TS_COL].diff().dropna()
    bad = g.loc[diffs != pd.Timedelta(hours=1), TS_COL].head(10).tolist()
    raise ValueError(f"Non-1h gaps remain around: {bad}")

#now use tqdm to iterate through a new forecast every following day starting at 2025-01-01 always at 8am and average out the RMSEs
from tqdm.notebook import tqdm
rmse_list = []
start_date = pd.Timestamp("2025-01-01 08:00:00")
# end_date = df[TS_COL].max() - pd.Timedelta(hours=8)
end_date = pd.Timestamp("2025-02-01 08:00:00")
for _ in tqdm(range((end_date - start_date).days)):
    print(start_date)
    history_df = df[df[TS_COL] <= start_date]
    forecast_df = pipeline.predict_df(
        history_df,
        prediction_length=8,
        quantile_levels=[0.5],
        id_column=ID_COL,
        timestamp_column=TS_COL,
        target=TARGETS,
    )
    actuals_df = df[(df[TS_COL] > start_date) & (df[TS_COL] <= start_date + pd.Timedelta(hours=8))]
    merged_df = pd.merge(forecast_df, actuals_df, on=[ID_COL, TS_COL], suffixes=('_pred', '_actual'))
    rmse = mean_squared_error(
        merged_df['total_tbs'],
        merged_df['predictions']
    )
    rmse_list.append(rmse)
    start_date += pd.Timedelta(days=1)
    #clear output
    clear_output(wait=True)
average_rmse = sum(rmse_list) / len(rmse_list)
print(f"Average RMSE over the period: {average_rmse}")

Average RMSE over the period: 78.64947186187389


In [21]:
#add holidays as a feature and see if it improves performance
import pandas as pd
import holidays

def add_holiday_flags(
    df: pd.DataFrame,
    ts_col: str = "ds",
    local_tz: str = "America/Montreal",
    observed: bool = True,
    include_names: bool = False,
) -> pd.DataFrame:
    """
    Adds boolean columns:
      • is_qc_holiday       — Québec public holiday (CA-QC)
      • is_jewish_holiday   — Israeli public/Jewish holiday (IL)
    Optionally adds:
      • qc_holiday_name
      • jewish_holiday_name

    Notes:
      • Holiday checks are date-based (00:00–24:00 local calendar date),
        not sundown-to-sundown observance.
      • NaT timestamps are ignored gracefully.
    """
    out = df.copy()

    # 1) Parse to datetime
    out[ts_col] = pd.to_datetime(out[ts_col], errors="coerce")

    # 2) Get the calendar DATE to use for holiday lookup
    #    - If tz-aware: convert to Montreal then take .date
    #    - If naive: assume values already represent local Montreal wall-clock; just take .date
    if getattr(out[ts_col].dt, "tz", None) is not None:
        dates_for_calendar = out[ts_col].dt.tz_convert(local_tz).dt.date
    else:
        dates_for_calendar = out[ts_col].dt.date

    # 3) Build a SAFE integer year range for the holiday objects
    years_series = pd.Series(dates_for_calendar)
    years_series = years_series.dropna().map(lambda d: int(pd.Timestamp(d).year))
    if years_series.empty:
        raise ValueError("No valid datetimes found to extract holiday years.")
    years = list(range(int(years_series.min()), int(years_series.max()) + 1))

    # 4) Construct holiday calendars
    qc_holidays = holidays.Canada(subdiv="QC", years=years, observed=observed)
    il_holidays = holidays.Israel(years=years, observed=observed)

    # 5) Flag membership
    # out["is_qc_holiday"] = [ (d in qc_holidays) if pd.notna(pd.Timestamp(d)) else False
    #                          for d in dates_for_calendar ]
    out["is_jewish_holiday"] = [ (d in il_holidays) if pd.notna(pd.Timestamp(d)) else False
                                 for d in dates_for_calendar ]

    if include_names:
        out["qc_holiday_name"] = [ qc_holidays.get(d, "") if pd.notna(pd.Timestamp(d)) else ""
                                   for d in dates_for_calendar ]
        out["jewish_holiday_name"] = [ il_holidays.get(d, "") if pd.notna(pd.Timestamp(d)) else ""
                                       for d in dates_for_calendar ]

    return out

df = pd.read_csv('https://www.dropbox.com/scl/fi/s83jig4zews1xz7vhezui/allDataWithCalculatedColumns.csv?rlkey=9mm4zwaugxyj2r4ooyd39y4nl&raw=1')
df.ds = pd.to_datetime(df.ds, errors="coerce")
df['id']='jgh'
df = add_holiday_flags(df, ts_col=TS_COL, include_names=False)

ID_COL = "id"
TS_COL = "ds"
TARGETS = ['total_tbs']

df = df.copy()
df[TS_COL] = pd.to_datetime(df[TS_COL], errors="coerce")
df = df.dropna(subset=[TS_COL])

# Snap to exact hours (lowercase 'h' to avoid FutureWarning)
df[TS_COL] = df[TS_COL].dt.floor("h")

# Sort + dedupe
df = df.sort_values([ID_COL, TS_COL]).drop_duplicates([ID_COL, TS_COL], keep="last")

def regularize_hourly(g: pd.DataFrame) -> pd.DataFrame:
    """
    Reindex each group's timestamps to strict hourly and fill gaps.
    Works whether the grouping column is present or omitted (include_groups=False).
    """
    # The group key (id) is available as g.name; if ID_COL exists, prefer it.
    sid = g[ID_COL].iloc[0] if ID_COL in g.columns else g.name

    g = g.sort_values(TS_COL)
    full_idx = pd.date_range(g[TS_COL].min(), g[TS_COL].max(), freq="h")
    g = g.set_index(TS_COL).reindex(full_idx)
    g.index.name = TS_COL

    # restore id (constant for the whole group)
    g[ID_COL] = sid

    # numeric + fill for targets
    for col in TARGETS:
        if col in g.columns:
            g[col] = pd.to_numeric(g[col], errors="coerce").ffill().bfill()
    return g.reset_index()

# Call apply with include_groups=False if supported; else fall back
gb = df.groupby(ID_COL, group_keys=False)
try:
    df = gb.apply(regularize_hourly, include_groups=False)
except TypeError:
    # older pandas without include_groups
    df = gb.apply(regularize_hourly)

# Assert truly hourly (accept 'h' and 'H')
g = df[df[ID_COL] == "jgh"].sort_values(TS_COL)
freq = pd.infer_freq(g[TS_COL])
if not freq:
    raise ValueError("No inferable frequency after regularization.")
if to_offset(freq).name.lower() != "h":
    # extra check independent of infer_freq
    diffs = g[TS_COL].diff().dropna()
    bad = g.loc[diffs != pd.Timedelta(hours=1), TS_COL].head(10).tolist()
    raise ValueError(f"Non-1h gaps remain around: {bad}")

#now use tqdm to iterate through a new forecast every following day starting at 2025-01-01 always at 8am and average out the RMSEs
from tqdm.notebook import tqdm
rmse_list = []
start_date = pd.Timestamp("2025-01-01 08:00:00")
# end_date = df[TS_COL].max() - pd.Timedelta(hours=8)
end_date = pd.Timestamp("2025-02-01 08:00:00")
for _ in tqdm(range((end_date - start_date).days)):
    print(start_date)
    history_df = df[df[TS_COL] <= start_date]
    forecast_df = pipeline.predict_df(
        history_df,
        prediction_length=8,
        quantile_levels=[0.5],
        id_column=ID_COL,
        timestamp_column=TS_COL,
        target=TARGETS,
    )
    actuals_df = df[(df[TS_COL] > start_date) & (df[TS_COL] <= start_date + pd.Timedelta(hours=8))]
    merged_df = pd.merge(forecast_df, actuals_df, on=[ID_COL, TS_COL], suffixes=('_pred', '_actual'))
    rmse = mean_squared_error(
        merged_df['total_tbs'],
        merged_df['predictions']
    )
    rmse_list.append(rmse)
    start_date += pd.Timedelta(days=1)
    #clear output
    clear_output(wait=True)
average_rmse = sum(rmse_list) / len(rmse_list)
print(f"Average RMSE over the period: {average_rmse}")

Average RMSE over the period: 78.5571038847945


In [34]:
df = pd.read_csv('https://www.dropbox.com/scl/fi/s83jig4zews1xz7vhezui/allDataWithCalculatedColumns.csv?rlkey=9mm4zwaugxyj2r4ooyd39y4nl&raw=1')
df.ds = pd.to_datetime(df.ds, errors="coerce")
df['id']='jgh'

hourly_shifts = pd.read_csv('https://www.dropbox.com/scl/fi/1ap6anzn5q13lcykh3idn/hourly_shifts.csv?rlkey=ue7nnfmrxij8keku8cgsrveo8&raw=1')
hourly_shifts.ds = pd.to_datetime(hourly_shifts.ds, errors="coerce")
#replace all NaNs with NoMD in hourly_shifts
hourly_shifts = hourly_shifts.fillna('NoMD')

df = pd.merge(df, hourly_shifts, on='ds', how='left')

ID_COL = "id"
TS_COL = "ds"
TARGETS = ['total_tbs']

df = df.copy()
df[TS_COL] = pd.to_datetime(df[TS_COL], errors="coerce")
df = df.dropna(subset=[TS_COL])

# Snap to exact hours (lowercase 'h' to avoid FutureWarning)
df[TS_COL] = df[TS_COL].dt.floor("h")

# Sort + dedupe
df = df.sort_values([ID_COL, TS_COL]).drop_duplicates([ID_COL, TS_COL], keep="last")

def regularize_hourly(g: pd.DataFrame) -> pd.DataFrame:
    """
    Reindex each group's timestamps to strict hourly and fill gaps.
    Works whether the grouping column is present or omitted (include_groups=False).
    """
    # The group key (id) is available as g.name; if ID_COL exists, prefer it.
    sid = g[ID_COL].iloc[0] if ID_COL in g.columns else g.name

    g = g.sort_values(TS_COL)
    full_idx = pd.date_range(g[TS_COL].min(), g[TS_COL].max(), freq="h")
    g = g.set_index(TS_COL).reindex(full_idx)
    g.index.name = TS_COL

    # restore id (constant for the whole group)
    g[ID_COL] = sid

    # numeric + fill for targets
    for col in TARGETS:
        if col in g.columns:
            g[col] = pd.to_numeric(g[col], errors="coerce").ffill().bfill()
    return g.reset_index()

# Call apply with include_groups=False if supported; else fall back
gb = df.groupby(ID_COL, group_keys=False)
try:
    df = gb.apply(regularize_hourly, include_groups=False)
except TypeError:
    # older pandas without include_groups
    df = gb.apply(regularize_hourly)

# Assert truly hourly (accept 'h' and 'H')
g = df[df[ID_COL] == "jgh"].sort_values(TS_COL)
freq = pd.infer_freq(g[TS_COL])
if not freq:
    raise ValueError("No inferable frequency after regularization.")
if to_offset(freq).name.lower() != "h":
    # extra check independent of infer_freq
    diffs = g[TS_COL].diff().dropna()
    bad = g.loc[diffs != pd.Timedelta(hours=1), TS_COL].head(10).tolist()
    raise ValueError(f"Non-1h gaps remain around: {bad}")

#now use tqdm to iterate through a new forecast every following day starting at 2025-01-01 always at 8am and average out the RMSEs
from tqdm.notebook import tqdm
rmse_list = []
start_date = pd.Timestamp("2025-01-01 16:00:00")
end_date = df[TS_COL].max() - pd.Timedelta(hours=8)
# end_date = pd.Timestamp("2025-02-01 08:00:00")
for _ in tqdm(range((end_date - start_date).days)):
    print(start_date)
    history_df = df[df[TS_COL] <= start_date]
    future_df = hourly_shifts[hourly_shifts['ds'] > start_date].head(8)
    future_df['id'] = 'jgh'
    forecast_df = pipeline.predict_df(
        history_df,
        future_df = future_df,
        prediction_length=8,
        quantile_levels=[0.5],
        id_column=ID_COL,
        timestamp_column=TS_COL,
        target=TARGETS,
    )
    actuals_df = df[(df[TS_COL] > start_date) & (df[TS_COL] <= start_date + pd.Timedelta(hours=8))]
    merged_df = pd.merge(forecast_df, actuals_df, on=[ID_COL, TS_COL], suffixes=('_pred', '_actual'))
    rmse = mean_squared_error(
        merged_df['total_tbs'],
        merged_df['predictions']
    )
    rmse_list.append(rmse)
    start_date += pd.Timedelta(days=1)
    #clear output
    clear_output(wait=True)
average_rmse = sum(rmse_list) / len(rmse_list)
print(f"Average RMSE over the period: {average_rmse}")

Average RMSE over the period: 57.56830870363711


In [ ]:
all_shifts_df = pd.read_csv('https://www.dropbox.com/scl/fi/yeyr2a7pj6nry8i2q3m0c/all_shifts.csv?rlkey=q1su2h8fqxfnlu7t1l2qe1w0q&raw=1')
all_shifts_df.head()

,scheduled_shift_id,group_id,user_id,employee_id,npi,first_name,last_name,facility_id,facility_ext_id,facility_name,...,shift_short_name,shift_start,shift_end,shift_hours,work_start,work_end,work_hours,count_as_shift,is_night,is_weekend
0,9021,1,40,NaN,NaN,Joel,Turner,1,NaN,Jewish General Hospital,...,W1,2021-01-01 08:00:00,2021-01-01 16:00:00,8.0,1/1/2021 08:00,1/1/2021 16:00,8.0,0,0,0
1,9022,1,52,NaN,NaN,Gregory,Marton,1,NaN,Jewish General Hospital,...,X1,2021-01-01 08:00:00,2021-01-01 17:00:00,9.0,1/1/2021 08:00,1/1/2021 17:00,9.0,1,0,0
2,9023,1,20,NaN,NaN,Will,Grad,1,NaN,Jewish General Hospital,...,X3,2021-01-01 08:00:00,2021-01-01 17:00:00,9.0,1/1/2021 08:00,1/1/2021 17:00,9.0,1,0,0
3,9025,1,17,NaN,NaN,Jerry,Dankoff,1,NaN,Jewish General Hospital,...,X4,2021-01-01 08:00:00,2021-01-01 16:00:00,8.0,1/1/2021 08:00,1/1/2021 16:00,8.0,1,0,0
4,9026,1,26,NaN,NaN,Marie Renee,Lajoie,1,NaN,Jewish General Hospital,...,X2,2021-01-01 08:00:00,2021-01-01 16:00:00,8.0,1/1/2021 08:00,1/1/2021 16:00,8.0,1,0,0


In [12]:
#make me a dataframe where each row is an hourly timestamp and columns for each user_id, with the value being the shift type for that user at that timestamp, and fill NaNs with NotWorking
shift_pivot = all_shifts_df.pivot(index='ds', columns='user_id', values='shift_type').fillna('NotWorking')
shift_pivot.head()

KeyError: 'ds'